In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

import matplotlib.pyplot as plt
%matplotlib inline

In [9]:
#create Spark session
spark = SparkSession.builder.appName('HW3').getOrCreate()

#change configuration settings on Spark 
conf = spark.sparkContext._conf.setAll([('spark.executor.memory', '5g'), ('spark.app.name', 'Spark Updated Conf'), ('spark.executor.cores', '4'), ('spark.cores.max', '4'), ('spark.driver.memory','8g')])

#print spark configuration settings
spark.sparkContext.getConf().getAll()

[('spark.eventLog.enabled', 'true'),
 ('spark.dynamicAllocation.minExecutors', '1'),
 ('spark.dataproc.sql.joinConditionReorder.enabled', 'true'),
 ('spark.history.fs.logDirectory',
  'gs://dataproc-temp-us-central1-360578915708-zif5fas7/69c1015c-6b85-43d5-bbaf-c3c1588d3cc3/spark-job-history'),
 ('spark.executor.memory', '5g'),
 ('spark.hadoop.mapreduce.fileoutputcommitter.concurrent.write.enabled',
  'false'),
 ('spark.yarn.historyServer.address', 'cluster-977d-m:18080'),
 ('spark.dataproc.sql.local.rank.pushdown.enabled', 'true'),
 ('spark.yarn.unmanagedAM.enabled', 'true'),
 ('spark.org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter.param.PROXY_HOSTS',
  'cluster-977d-m.us-central1-c.c.bdp-summer.internal.'),
 ('spark.ui.filters',
  'org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter'),
 ('spark.app.id', 'application_1689896722086_0004'),
 ('spark.metrics.namespace',
  'app_name:${spark.app.name}.app_id:${spark.app.id}'),
 ('spark.dataproc.sql.optimizer.leftsemijoin.

In [10]:
df = spark.read \
    .option("quote", "\"")  \
    .option("escape", "\"") \
    .option("ignoreLeadingWhiteSpace",True) \
    .csv("gs://bdp-hw-su/notebooks/data/Crimes_-_2001_to_Present.csv",inferSchema=True, header=True)

In [11]:
df.take(1)

[Row(ID=11646166, Case Number='JC213529', Date='09/01/2018 12:01:00 AM', Block='082XX S INGLESIDE AVE', IUCR='0810', Primary Type='THEFT', Description='OVER $500', Location Description='RESIDENCE', Arrest=False, Domestic=True, Beat=631, District=6, Ward=8, Community Area=44, FBI Code='06', X Coordinate=None, Y Coordinate=None, Year=2018, Updated On='04/06/2019 04:04:43 PM', Latitude=None, Longitude=None, Location=None)]

In [12]:
spark.stop()

23/07/21 00:32:08 ERROR YarnClientSchedulerBackend: The YARN application has already ended! It might have been killed or the Application Master may have failed to start. Check the YARN application logs for more details.
23/07/21 00:32:08 ERROR SparkContext: Error initializing SparkContext.
org.apache.spark.SparkException: Unmanaged application application_1689896722086_0003 failed due to ApplicationMaster for attempt appattempt_1689896722086_0003_000001 timed out. Failing the application.
	at org.apache.spark.scheduler.cluster.YarnClientSchedulerBackend.waitForApplication(YarnClientSchedulerBackend.scala:98) ~[spark-yarn_2.12-3.3.0.jar:3.3.0]
	at org.apache.spark.scheduler.cluster.YarnClientSchedulerBackend.start(YarnClientSchedulerBackend.scala:65) ~[spark-yarn_2.12-3.3.0.jar:3.3.0]
	at org.apache.spark.scheduler.TaskSchedulerImpl.start(TaskSchedulerImpl.scala:222) ~[spark-core_2.12-3.3.0.jar:3.3.0]
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:585) ~[spark-core_2.12-3.3